In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

# Scalar data assimilation
## Concepts
The following example uses our temperature measurements from room 2414 in the KB building to estimate the average temperature in the room. We operate with four(!) different expressions for the temperature: 

1. $T_t$ - the _true_ average temperature, which we will never know!
1. $T_m$ - the modeled average temperature, which we obtain from some kind of model (to be detailed later on).
1. $T_o$ - the observed temperature, using a combination of readings from our different thermometers. 
1. $T_a$ - the analysed temperature, which is a combination of the model and observations.

We require that the **observations and the model are free from biases**. The so-called analysis equation is

$$T_a = T_m + K(T_o - T_m),$$

with 

$$K = \frac{e_m^2}{e_m^2 + e_o^2}.$$

Here $e_m^2 = <(T_m-T_t)^2>$ and $e_o^2 = <(T_o-T_t)^2>$ are the expected model and observation error variances, i.e. the notation "$<\,\,\,>$" implies an averaging procedure. Recall that we require that the **observation errors and model errors are uncorrelated**. Think about this statement. We do not say that the observations and the model have to be completely unrelated, we just say that their errors cannot have mutual dependencies. Also, since we do not know the true average temperature $T_t$, we will need to estimate the errors. 



## Procedure
What we have done so far: 

1. We have made simultaneous measurements $T_i$ with all instruments for the purpose of intercalibration, 
1. we have made temperature readings with all instruments placed in different places in the room, and 
1. we have received no feedback from the technical dept., hence we have really no idea how the room temperature is controlled through the ventilation system, and have no objective information on which we can build a model for the average room temperature. 

We will now 

1. start with intercalibrating the thermometers and correcting the readings, 
1. devise observation error estimates, 
1. present a model, with an associated model error, 
1. insert all the values we need into the analysis equation, and, finally
1. calculate the analysis error.

In [2]:
# Intercalibration readings
T_i_uncalibrated = [20.2, 20.2, 19.0, 19.5]
T_i_avg = np.mean(T_i_uncalibrated) # Mean of readings
T_bias = T_i_uncalibrated - T_i_avg # The bias is here defined as the deviation from the average 

# Correcting the readings made in the different parts of the room
T_o_uncalibrated = [20.6, 20.4, 20.0, 19.5]
T_o_calibrated = T_o_uncalibrated - T_bias

# Calculating the bias-corrected observation average
T_o = np.average(T_o_calibrated)

## Observation error
The observation error consists of two parts. The first part, the instrument error, $e_{oi}$, is easy to understand, but for us rather hard to estimate. This is simply the error in the measurement due to hardware and/or operator error, which should properly be checked comparing with a very high quality thermometer. We haven't got such high quality thermometer, hence we will simply have to make a qualified guess. Typically, a cheap thermometer has a resolution of 1 degree, and we can e.g. assume that the instrument error is about half of that, especially if the person reading off the values is nearing his fifties and not wearing his reading glasses. 

The most interesting error is the so-called representation error, $e_{or}$. Recall that we want to know the average temperature in the room. If we put our thermometers in awkward places, e.g. close to the door or the windows, on the floor etc., we wouldn't expect the readings to be representative for the average room temperature, regardless of the quality of the thermometer. The model simply cannot represent the more complex processes observed by the instrument. 

How can we estimate the representation error, then? Since we have several readings, we can take their standard deviation and use that. What we are doing is so-called "super-obing". We have strictly speaking more observations than we need - the observations have higher resolution than the model - and when we average them we get a better handle on the representation error. The use of super-obing is not uncommon in real-world data assimilative forecasting systems. The opposite situation, when the observations have lower resolution than the model, is called "super-modding". Then model values are averaged (e.g. over several grid cells) before comparison with the observations. One example is SST observed using satellites with passive microwave sensors, which typically have very coarse spatial resolution, but on the positive side have the ability to penetrate the cloud cover.  

In [3]:
# Instrument error
e_oi = 0.5

# Representation error
e_or = np.std(T_o_calibrated)

# Combined error variance
e_o2 = (e_oi + e_or)**2 

## Model and model error
The model proposed here is to simply to assume that the temperature is constant, and to find the value of this constant we take the average of the bias-corrected readings we took for intercalibration. The motivation for this choice is that 

1. we have no independent information about how the room temperature is regulated, 
1. the intercalibration readings were made approximately in the middle of the room, and hence (hopefully) provide a reasonable representation of the room average, and
1. the readings were made independently of the readings that will be used in the analysis equation.

Now we need to find a model error estimate. Depending on the weather and the season, I suppose it is reasonable to assume that the temperature will vary within a range of 4-5 degrees, say. Assuming that we have happily selected a model that is bias free, the model error would then be up to 2.5 degrees.

In [4]:
# Model error variance
T_m = T_i_avg
e_m2 = 2.5**2

## Analysis and analysis error
It now remains to insert the observation and model values and errors into the analysis equation. Also, recall that the analysis equation can be used to estimate the analysis error:

$$e_a^2 = (1-K)^2 e_m^2 + K^2 e_o^2$$

After all, the value of $K$ is obtained requiring

$$\frac{d e_a^2}{dK} = 0,$$

since we then have a minimum in the analysis error.

In [5]:
# Analysis
K = e_m2/(e_m2 + e_o2)
T_a = T_m + K*(T_o - T_m)
e_a2 = (1-K)*e_m2 + K*e_o2

# Output
print('The modeled average room temperature is %5.3f C.' % T_m)
print('The observed average room temperature is %5.3f C.' % T_o)
print('The analysed average room temperature is %5.3f C.' % T_a)
print('The analysis error is %5.3f C.' % np.sqrt(e_a2))

The modeled average room temperature is 19.725 C.
The observed average room temperature is 20.125 C.
The analysed average room temperature is 20.081 C.
The analysis error is 1.167 C.


## Discussion points

1. Is the model bias free?
1. Is the observation bias free?
1. Are the model and observation errors uncorrelated? 
1. Is the estimate of the instrument error reasonable?